In [1]:
import requests
from pathlib import Path
import pandas as pd

In [2]:
# create directory
Path("../Data/TenderNed").mkdir(parents=True, exist_ok=True)

In [3]:
url = "https://www.tenderned.nl/cms/sites/default/files/2024-02/2023.json"

with requests.session() as session:
    # download the file
    r = session.get(url, allow_redirects=True, stream=True)

    # save the file
    target_path = str('../Data/TenderNed/TenderNed_2023.json')
    with open(target_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=10 * 1024):
            f.write(chunk)

In [4]:
# laod the file
import json
with open('../Data/TenderNed/TenderNed_2023.json') as f:
    data = json.load(f)    

In [5]:
# convert releases from data to dataframe
df = pd.DataFrame(data['releases'])

In [6]:
# normalize the json data
df2 = pd.json_normalize(df["tender"])
df2.columns = [f"tender_{col}" for col in df2.columns]
df3 = pd.json_normalize(df["buyer"])
df3.columns = [f"buyer_{col}" for col in df3.columns]
df4 = pd.json_normalize(df["parties"])
df4.columns = [f"parties_{col}" for col in df4.columns]
# drop columns that are not needed
df = df.drop(columns=["tender", "buyer", "parties"])
df6 = pd.concat([df, df2, df3, df4], axis=1)

# define columns to be embedded and columns to be added to the metadata
columns_to_embed = ["tender_description"]
columns_to_metadata = ["ocid", "date", "initiationType", "tender_title", "tender_documents", "tender_nationalOrEuropean", "tender_noticeTypeDetails", "tender_mainProcurementCategory", "tender_awardCriteriaDetails", "tender_tenderPeriod.endDate", "tender_contractPeriod.endDate", "tender_classification.description", "tender_value.amount", "tender_lots"]

In [7]:
# write to csv
df6.to_csv("../Data/TenderNed/TenderNed_2023.csv", index=False)